In [1]:
import os
import json
import pandas as pd
import duckdb
from pydantic import BaseModel, Field
from openai import OpenAI
from dotenv import load_dotenv

# Observability and Tracing imports (Phoenix)
import phoenix as px
from openinference.instrumentation.openai import OpenAIInstrumentor
from openinference.instrumentation import TracerProvider
from phoenix.otel import register
from opentelemetry.trace import StatusCode
import nest_asyncio

# Load .env variables immediately
load_dotenv()

# Apply asyncio patch
nest_asyncio.apply()

# ==============================================================================
# 1. CONFIGURATION & SETUP
# ==============================================================================

MODEL = "gpt-4o-mini"
# 🔄 CHANGE 1: New file name
TRANSACTION_DATA_FILE_PATH = 'employee_data.parquet'


# API Key Handling
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ API Key missing! Please set 'OPENAI_API_KEY' in your .env file.")

client = OpenAI(api_key=api_key)
print("✅ OpenAI Client Initialized successfully.")

# ==============================================================================
# 2. PHOENIX TRACING SETUP
# ==============================================================================
PROJECT_NAME = "employee-analysis-agent"
os.environ["PHOENIX_PROJECT_NAME"] = PROJECT_NAME

# Ensure Phoenix server is running on port 6006
tracer_provider = register(
    project_name=os.environ["PHOENIX_PROJECT_NAME"], # Best practice to reference the env var
    endpoint="http://localhost:6006/v1/traces"
)

OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)
tracer = tracer_provider.get_tracer(__name__)




# ==============================================================================
# 3. TOOL DEFINITIONS
# ==============================================================================

# --- TOOL 1: Database Lookup (Text-to-SQL) ---
SQL_GENERATION_PROMPT = """
Generate an SQL query based on a prompt. Do not reply with anything besides the SQL query.
The prompt is: {prompt}

The available columns are: {columns}
The table name is: {table_name}
"""

def generate_sql_query(prompt: str, columns: list, table_name: str) -> str:
    formatted_prompt = SQL_GENERATION_PROMPT.format(prompt=prompt, columns=columns, table_name=table_name)
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": formatted_prompt}],
    )
    return response.choices[0].message.content

# 🔄 CHANGE 2: Renamed function and updated table logic
@tracer.tool() 
def lookup_employee_data(prompt: str) -> str:
    """
    Tool Logic:
    1. Reads employee parquet file.
    2. SQL Generation via DuckDB.
    """
    try:
        # 🔄 CHANGE 3: Table name is now 'employees'
        table_name = "employees"
        
        # Load data
        df = pd.read_parquet(TRANSACTION_DATA_FILE_PATH)
        
        # Register the dataframe as a SQL table in DuckDB
        duckdb.sql(f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM df")

        # Step 2: Generate SQL
        sql_query = generate_sql_query(prompt, list(df.columns), table_name)
        
        # Cleaning: Remove markdown code blocks
        sql_query = sql_query.strip().replace("```sql", "").replace("```", "")
        
        # Step 3: Execute SQL with tracing
        with tracer.start_as_current_span("execute_sql_query") as span:
            result = duckdb.sql(sql_query).df()
            span.set_attribute("sql_query", sql_query)
            span.set_status(StatusCode.OK)
        
        return result.to_string()
        
    except Exception as e:
        return f"Error accessing data: {str(e)}"


# --- TOOL 2: Data Analysis ---
DATA_ANALYSIS_PROMPT = """
Analyze the following data: {data}
Your job is to answer the following question: {prompt}
"""

@tracer.tool()
def analyze_data(prompt: str, data: str) -> str:
    """Derive insights from raw data."""
    formatted_prompt = DATA_ANALYSIS_PROMPT.format(data=data, prompt=prompt)

    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": formatted_prompt}],
    )
    
    analysis = response.choices[0].message.content
    return analysis if analysis else "No analysis could be generated"


# --- TOOL 3: Data Visualization ---
class VisualizationConfig(BaseModel):
    chart_type: str = Field(..., description="Type of chart (bar, line, scatter)")
    x_axis: str = Field(..., description="Column name for x-axis")
    y_axis: str = Field(..., description="Column name for y-axis")
    title: str = Field(..., description="Chart title")

@tracer.chain()
def extract_chart_config(data: str, visualization_goal: str) -> dict:
    prompt = f"Generate chart config for this data based on goal: {visualization_goal}\nData: {data}"
    
    response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        response_format=VisualizationConfig,
    )
    
    content = response.choices[0].message.parsed
    return {
        "chart_type": content.chart_type,
        "x_axis": content.x_axis,
        "y_axis": content.y_axis,
        "title": content.title,
        "data_snippet": data[:100] 
    }

CREATE_CHART_PROMPT = """
Write python code (using matplotlib/seaborn) to create a chart based on this config.
Only return the code, no markdown.
config: {config}
"""

@tracer.chain()
def create_chart_code(config: dict) -> str:
    formatted_prompt = CREATE_CHART_PROMPT.format(config=config)
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": formatted_prompt}],
    )
    code = response.choices[0].message.content
    return code.strip().replace("```python", "").replace("```", "")

@tracer.tool()
def generate_visualization(data: str, visualization_goal: str) -> str:
    config = extract_chart_config(data, visualization_goal)
    code = create_chart_code(config)
    return f"Generated Python Code for Chart:\n{code}"


# ==============================================================================
# 4. TOOL SCHEMA & MAPPING
# ==============================================================================
tool_implementations = {
    "lookup_employee_data": lookup_employee_data,
    "analyze_data": analyze_data,
    "generate_visualization": generate_visualization
}

# 🔄 CHANGE 4: Updated Schema descriptions
tools_schema = [
    {
        "type": "function",
        "function": {
            "name": "lookup_employee_data",
            "description": "Query the Employee SQL database for info on salaries, departments, and performance.",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {"type": "string", "description": "The question to ask the database (e.g. 'average salary by dept')"}
                },
                "required": ["prompt"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_data",
            "description": "Analyze text or tabular data to find trends.",
            "parameters": {
                "type": "object",
                "properties": {
                    "data": {"type": "string", "description": "The data found from lookup_employee_data"},
                    "prompt": {"type": "string", "description": "Specific question about the data"}
                },
                "required": ["data", "prompt"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_visualization",
            "description": "Generate Python code to plot data.",
            "parameters": {
                "type": "object",
                "properties": {
                    "data": {"type": "string", "description": "The data to plot"},
                    "visualization_goal": {"type": "string", "description": "What the chart should show"}
                },
                "required": ["data", "visualization_goal"]
            }
        }
    }
]


# ==============================================================================
# 5. AGENT ORCHESTRATION LOOP
# ==============================================================================

@tracer.chain()
def handle_tool_calls(tool_calls, messages):
    for tool_call in tool_calls:
        fn_name = tool_call.function.name
        fn_args = json.loads(tool_call.function.arguments)
        
        print(f"🛠️ Agent is calling tool: {fn_name}")
        
        function_to_call = tool_implementations.get(fn_name)
        
        if function_to_call:
            result = function_to_call(**fn_args)
            messages.append({
                "role": "tool", 
                "content": str(result), 
                "tool_call_id": tool_call.id
            })
        else:
            print(f"Error: Tool {fn_name} not found.")
            
    return messages

def run_agent(user_query: str):
    print(f"\n🤖 Agent starting with query: {user_query}")
    
    messages = [
        {"role": "system", "content": "You are a helpful HR data analyst agent. Use the available tools to answer questions."},
        {"role": "user", "content": user_query}
    ]

    while True:
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools_schema,
        )

        ai_message = response.choices[0].message
        messages.append(ai_message.model_dump())

        if ai_message.tool_calls:
            messages = handle_tool_calls(ai_message.tool_calls, messages)
        else:
            print("\n✅ Agent finished.")
            return ai_message.content


# ==============================================================================
# 6. MAIN EXECUTION
# ==============================================================================
def create_dummy_data():
    """Generates a dummy parquet file for HR/Employee data."""
    if not os.path.exists(TRANSACTION_DATA_FILE_PATH):
        print("Creating dummy employee data...")
        # 🔄 CHANGE 5: New Data Structure
        data = {
            'employee_id': [101, 102, 103, 104, 105, 106, 107, 108],
            'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Evan', 'Fiona', 'George', 'Hannah'],
            'department': ['Engineering', 'Sales', 'Engineering', 'HR', 'Sales', 'Engineering', 'Marketing', 'Engineering'],
            'salary': [120000, 85000, 135000, 70000, 95000, 110000, 88000, 140000],
            'years_experience': [5, 3, 8, 2, 5, 4, 3, 10],
            'performance_score': [4.5, 4.0, 4.8, 3.9, 4.2, 3.5, 4.1, 4.9]
        }
        df = pd.DataFrame(data)
        df.to_parquet(TRANSACTION_DATA_FILE_PATH)
        print(f"Saved to {TRANSACTION_DATA_FILE_PATH}")

if __name__ == "__main__":
    create_dummy_data()
    
    # 🔄 UPDATED QUERY: Forces the agent to "Analyze" first
    QUERY = (
        "First, get the performance scores and salaries for the Engineering department. "
        "Then, analyze the data to tell me if higher pay correlates with better performance. "
        "Finally, create a visualization code to prove your point."
    )
    
    final_response = run_agent(QUERY)
    print("\n------------------------------------------------")
    print("FINAL RESPONSE:")
    print(final_response)
    print("------------------------------------------------")
    print(f"View traces at: http://localhost:6006")

✅ OpenAI Client Initialized successfully.
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: employee-analysis-agent
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.


🤖 Agent starting with query: First, get the performance scores and salaries for the Engineering department. Then, analyze the data to tell me if higher pay correlates with better performance. Finally, create a visualization code to prove your point.
🛠️ Agent is calling tool: lookup_employee_data
🛠️ Agent is calling tool: analyze_data
🛠️ Agent is calling tool: generate_vis